In [4]:
import os
import argparse

import pickle
import time

import neat
# import visualize
# import reporters as r

import multiprocessing
import pathlib
import numpy as np

from objects import BallOnPlate

In [5]:
simulation_seconds = 20.
    
def eval_genome(genome, config):
    ballOnPlate = BallOnPlate(showGUI=False, randomInitial=False)

    net = neat.nn.FeedForwardNetwork.create(genome, config)

    global_cost = 1e10

    CONST_VALUE = 0.7
    intial_positions = [
        [CONST_VALUE, CONST_VALUE],
        [-CONST_VALUE, -CONST_VALUE],
        [-CONST_VALUE, CONST_VALUE],
        [CONST_VALUE, -CONST_VALUE],
#         [0., 0.]
    ]

    reference_positions = [
        [-CONST_VALUE, -CONST_VALUE],
        [CONST_VALUE, CONST_VALUE],
        [CONST_VALUE, -CONST_VALUE],
        [-CONST_VALUE, CONST_VALUE],
#         [0., 0.]
    ]

    for i in range(len(intial_positions)):

        reference_point = np.array(reference_positions[i])
        initial_position = np.array(intial_positions[i])
        
#         while abs(length_rate) < 0.4:
#             reference_point = np.random.randint(-9, 9, 2) / 10.
#             initial_position = np.random.randint(-9, 9, 2) / 10.


#     #         dx = reference_positions[i][0] - intial_positions[i][0]
#     #         dy = reference_positions[i][1] - intial_positions[i][1]
        length_rate = np.linalg.norm(
            np.array(reference_point) - np.array(initial_position)
        )
        
        result = 0
        dropDown = False

        ref_point = np.array(reference_point)

        posOnPlate = ballOnPlate.reset(initial_position)
        prevPosOnPlate = posOnPlate

        prev_err    = np.array([0, 0])
        integr_err  = [0, 0]
        control     = np.array([0, 0])
        
        while ballOnPlate.time < simulation_seconds:
            # half of plate circle
            # if i == 4:
                # ref_point = np.array([.5*math.cos(ballOnPlate.time/2), .5*math.sin(ballOnPlate.time/2)])
            # elif i == 5:
                # ref_point = np.array([.5*math.cos(ballOnPlate.time), .5*math.sin(ballOnPlate.time)])

            # Get error
            err = ref_point - posOnPlate
            result -= np.sqrt(err[0] * err[0] + err[1] * err[1]) * (ballOnPlate.time + 1) / length_rate

            speed = (posOnPlate - prevPosOnPlate) #/BallOnPlate.D_T

            # Process control system
            netInput = np.array([err[0], err[1], 
                                 posOnPlate[0], posOnPlate[1], 
                                 control[0], control[1], 
                                 speed[0], speed[1]])
            # print(netInput)
            netOutput = net.activate(netInput)

            ### PID controller
            prop    = netOutput[0]
            diff    = netOutput[1] / BallOnPlate.D_T
            integr  = netOutput[2] * BallOnPlate.D_T
                
            integr_err += err
            d_err = err - prev_err

            control = prop * err + diff * d_err + integr_err * integr
            control = np.array([-control[1], control[0]])
            
#             envInput[0] = prop * err[1] + diff * d_err[1] + integr_err[1] * integr
#             envInput[0] = -envInput[0]
#             envInput[1] = prop * err[0] + diff * d_err[0] + integr_err[0] * integr

            prev_err = err
            ### PID controller

            prevPosOnPlate = posOnPlate

            control = np.clip(control, -1, 1)
            
            posOnPlate, isEnd = ballOnPlate.step(control)
            if isEnd:
                # Bad penalty as fall
                dropDown = True
                break
            # sleep(ballOnPlate.dt)

        DROP_PENALTY_VALUE = 1e3

        if dropDown:
            current_cost = ballOnPlate.time * (DROP_PENALTY_VALUE/(2*simulation_seconds)) + result - DROP_PENALTY_VALUE
        else:
            current_cost = ballOnPlate.time * (DROP_PENALTY_VALUE/(2*simulation_seconds)) + result

#         cost += current_cost / float(len(intial_positions))
        global_cost = min(current_cost, global_cost)

    ballOnPlate.close()
    return global_cost / 100
        
def eval_genomes(genomes, config):

    for genome_id, genome in genomes:
        genome.fitness = eval_genome(genome, config)

In [ ]:
populationCount     = 1000
numCores            = multiprocessing.cpu_count()
print('Processing {} cores'.format(numCores))

checkpointDir       = 'checkpoints_ff'
winnerFname         = 'winner_ff'
pictureDir          = 'pictures_ff'
configFname         = 'feedforward.cfg'
logFname            = 'log_ff.txt'

checkpointPrefix    = checkpointDir + '/chk_'

if not os.path.exists(checkpointDir):
    os.makedirs(checkpointDir)

# if not os.path.exists(pictureDir):
#     os.makedirs(pictureDir)

    
# if restoreCheckpoint is None:
local_dir = str(pathlib.Path().absolute())
config_path = os.path.join(local_dir, configFname)
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)

pop = neat.Population(config)
# else:
#     pop = neat.Checkpointer.restore_checkpoint(restoreCheckpoint)

stats = neat.StatisticsReporter()
pop.add_reporter(stats)
pop.add_reporter(neat.StdOutReporter(True))
# pop.add_reporter(r.FileReporter(logFname, True))
pop.add_reporter(neat.Checkpointer(generation_interval=100, filename_prefix=checkpointPrefix))

# if render_flag:
# winner = pop.run(eval_genomes)
# else:
pe = neat.ParallelEvaluator(numCores, eval_genome)
winner = pop.run(pe.evaluate, n=populationCount)

# Save the winner.
with open(winnerFname, 'wb') as f:
    pickle.dump(winner, f)

# visualize.plot_stats(stats, view=False, ylog=True, filename=pictureDir+'/fitness.svg')
# visualize.plot_species(stats, view=False, filename=pictureDir+'/speciation.svg')

# node_names = {-1: 'ext', -2: 'eyt', -3: 'sf', -4: 'sl', -5: 'sr', -6: 'sb', 0: 'ux', 1: 'uy'}
# visualize.draw_net(config, winner, False, node_names=node_names,
                   # filename='pictures_ff/Digraph.gv')
# visualize.draw_net(config, winner, view=False, node_names=node_names,
                   # filename="pictures_ff/winner-feedforward.gv")
# visualize.draw_net(config, winner, view=False, node_names=node_names,
                   # filename="pictures_ff/winner-feedforward-enabled.gv", show_disabled=False)
#visualize.draw_net(config, winner, view=False, node_names=node_names,
#                   filename="winner-feedforward-enabled-pruned.gv", show_disabled=False, prune_unused=True)


Population's average fitness: -20.05048 stdev: 30.33384
Best fitness: -2.67493 - size: (4, 21) - species 3 - id 1217
Average adjusted fitness: 0.819
Mean genetic distance 2.630, standard deviation 0.545
Population of 153 members in 25 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   13    11    -10.3    0.943     3
     2   13     2   -136.4    0.000    12
     3   13     8     -2.7    0.885     5
     4   12    10    -10.3    0.879     0
     5   12     8    -10.3    0.943     4
     6   12     8    -10.3    0.943     0
     7   11     1    -10.3    0.943     5
     8   11     8    -10.3    0.756     4
     9   11    10    -10.3    0.918     0
    10    9     7    -10.3    0.848     3
    11    8     8    -10.3    0.844     1
    12    8     3    -10.3    0.817     0
    13    7     4     -4.3    0.732     5
    14    6     8    -10.3    0.920     0
    15    6     9    -10.3    0.943     0
    16    5     8    -10.3    0.943     0
